# Chessboards and knight's tours in Raku

<span style="font-size: 16pt; font-style: italic; font-weight: bold">...via graph functionalities</span>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)      
November 2024

-----

## Introduction

- Two ways of plotting chessboards
- [Knight's tour](https://en.wikipedia.org/wiki/Knight%27s_tour) graphs
- [Hamiltonian paths](https://en.wikipedia.org/wiki/Hamiltonian_path)

------

## Setup

Here are loaded the packages used in the rest of notebook:

In [1]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::HexagonalGrid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Nested;
use Graph::Path;
use Graph::Petersen;
use Graph::Star;
use Graph::TriangularGrid;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;

use Math::DistanceFunctions;
use Math::Nearest;
use Text::Levenshtein::Damerau;

use Hash::Merge;
use FunctionalParsers;
use FunctionalParsers::EBNF;
use EBNF::Grammar;
use Graphviz::DOT::Grammar;

use JavaScript::D3;
use WWW::MermaidInk;

use paths;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [4]:
my $title-color = 'Ivory';
my $tick-labels-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '#1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $engine = 'neato';
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

{background => #1F1F1F, edge-thickness => 3, title-color => Ivory, vertex-size => 6}

------

## Chessboards

Make a chessboard SVG plot using a grid graph:

In [5]:
#%html
my $gg = Graph::Grid.new(4, 6);

# Leveraging the grid graphs a bipartite
my %highlight = <DimGray LightGray> Z=> $gg.bipartite-coloring.classify(*.value).nodemap(*».key).values;


# DOT language spec
my $preamble = q:to/END/;
fontcolor = "Ivory";
fontsize = "16";
labelloc = "t";
label = "Chessboard by DOT";
graph [size="8,16!"];

bgcolor="#1F1F1F";
node [style=filled, label="", opacity=0.3, fixedsize=true, shape=square, color="Black", fillcolor="SlateBlue", penwidth=1, fontsize=4, fontcolor="White", labelloc=c, width=0.98, height=0.3];
edge [style=invis, color="SteelBlue", penwidth=0.6];
END

# To be reused for overlaying tour paths
my $dot = $gg.dot(:$preamble, :%highlight, :!node-labels).lines[1..*-2].join("\n");

# Rendering of the DOT spec
$gg.dot(:$preamble, :%highlight, :!node-labels, engine=>'neato'):svg;

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
 Chessboard by DOT 
<!-- 0_2 -->
 
 0_2 
 
 
<!-- 0_3 -->
 
 0_3 
 
 
<!-- 0_2--0_3 -->
<!-- 1_2 -->
 
 1_2 
 
 
<!-- 0_2--1_2 -->
<!-- 2_4 -->
 
 2_4 
 
 
<!-- 3_4 -->
 
 3_4 
 
 
<!-- 2_4--3_4 -->
<!-- 2_5 -->
 
 2_5 
 
 
<!-- 2_4--2_5 -->
<!-- 0_4 -->
 
 0_4 
 
 
<!-- 0_5 -->
 
 0_5 
 
 
<!-- 0_4--0_5 -->
<!-- 1_4 -->
 
 1_4 
 
 
<!-- 0_4--1_4 -->
<!-- 2_2 -->
 
 2_2 
 
 
<!-- 3_2 -->
 
 3_2 
 
 
<!-- 2_2--3_2 -->
<!-- 2_3 -->
 
 2_3 
 
 
<!-- 2_2--2_3 -->
<!-- 1_5 -->
 
 1_5 
 
 
<!-- 1_5--2_5 -->
<!-- 3_3 -->
 
 3_3 
 
 
<!-- 3_2--3_3 -->
<!-- 3_1 -->
 
 3_1 
 
 
<!-- 3_1--3_2 -->
<!-- 2_1 -->
 
 2_1 
 
 
<!-- 2_1--2_2 -->
<!-- 2_1--3_1 -->
<!-- 0_5--1_5 -->
<!-- 2_3--2_4 -->
<!-- 2_3--3_3 -->
<!-- 3_0 -->
 
 3_0 
 
 
<!-- 3_0--3_1 -->
<!-- 0_3--0_4 -->
<!-- 1_3 -->
 
 1_3 
 
 
<!-- 0_3--1_3 -->
<!-- 0_0 -->
 
 0_0 
 
 
<!-- 0_1 -->
 
 0_1 
 
 
<!-- 0_0--0_1 -->
<!-- 1_0 -->
 
 1_0 
 
 
<!-- 0_0--1_0 -->
<!-- 1_4--2_4 -->
<!-- 1_4--1_5 -->
<!-- 3_5 -->
 
 3_5 
 
 
<!-- 0_1--0_2 -->
<!-- 1_1 -->
 
 1_1 
 
 
<!-- 0_1--1_1 -->
<!-- 3_4--3_5 -->
<!-- 2_0 -->
 
 2_0 
 
 
<!-- 2_0--2_1 -->
<!-- 2_0--3_0 -->
<!-- 1_0--2_0 -->
<!-- 1_0--1_1 -->
<!-- 1_2--2_2 -->
<!-- 1_2--1_3 -->
<!-- 1_3--2_3 -->
<!-- 1_3--1_4 -->
<!-- 1_1--2_1 -->
<!-- 1_1--1_2 -->
<!-- 3_3--3_4 -->
<!-- 2_5--3_5 -->

Here is another way to make a chessboard (not used here because it is hard to do the combined board-tour plot below):

In [6]:
#% js
js-d3-chessboard('8/8/8/8/8/8/8/8', :$background, :$tick-labels-color, :$title-color, color-palette => 'Greys')

--------

## Knight's tour over a chessboard

Make a Knight's tour graph:

In [7]:
my $gkt = Graph::KnightTour.new(4, 6, prefix => 'kt');

Graph(vertexes => 24, edges => 44, directed => False)

Show the corresponding Graphviz DOT plot:

In [8]:
#% html
$gkt.dot(:$engine, :$background, vertex-shape => 'ellipse', vertex-width => 0.6):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- kt3_1 -->
 
 kt3_1 
 
 kt3_1 
 
<!-- kt5_2 -->
 
 kt5_2 
 
 kt5_2 
 
<!-- kt3_1--kt5_2 -->
 
 kt3_1--kt5_2 
 
 
<!-- kt4_3 -->
 
 kt4_3 
 
 kt4_3 
 
<!-- kt3_1--kt4_3 -->
 
 kt3_1--kt4_3 
 
 
<!-- kt5_0 -->
 
 kt5_0 
 
 kt5_0 
 
<!-- kt3_1--kt5_0 -->
 
 kt3_1--kt5_0 
 
 
<!-- kt0_1 -->
 
 kt0_1 
 
 kt0_1 
 
<!-- kt2_0 -->
 
 kt2_0 
 
 kt2_0 
 
<!-- kt0_1--kt2_0 -->
 
 kt0_1--kt2_0 
 
 
<!-- kt2_2 -->
 
 kt2_2 
 
 kt2_2 
 
<!-- kt0_1--kt2_2 -->
 
 kt0_1--kt2_2 
 
 
<!-- kt1_3 -->
 
 kt1_3 
 
 kt1_3 
 
<!-- kt0_1--kt1_3 -->
 
 kt0_1--kt1_3 
 
 
<!-- kt1_1 -->
 
 kt1_1 
 
 kt1_1 
 
<!-- kt2_3 -->
 
 kt2_3 
 
 kt2_3 
 
<!-- kt1_1--kt2_3 -->
 
 kt1_1--kt2_3 
 
 
<!-- kt3_2 -->
 
 kt3_2 
 
 kt3_2 
 
<!-- kt1_1--kt3_2 -->
 
 kt1_1--kt3_2 
 
 
<!-- kt3_0 -->
 
 kt3_0 
 
 kt3_0 
 
<!-- kt1_1--kt3_0 -->
 
 kt1_1--kt3_0 
 
 
<!-- kt1_0 -->
 
 kt1_0 
 
 kt1_0 
 
<!-- kt1_0--kt3_1 -->
 
 kt1_0--kt3_1 
 
 
<!-- kt1_0--kt2_2 -->
 
 kt1_0--kt2_2 
 
 
<!-- kt3_3 -->
 
 kt3_3 
 
 kt3_3 
 
<!-- kt3_3--kt5_2 -->
 
 kt3_3--kt5_2 
 
 
<!-- kt4_1 -->
 
 kt4_1 
 
 kt4_1 
 
<!-- kt3_3--kt4_1 -->
 
 kt3_3--kt4_1 
 
 
<!-- kt4_0 -->
 
 kt4_0 
 
 kt4_0 
 
<!-- kt4_0--kt5_2 -->
 
 kt4_0--kt5_2 
 
 
<!-- kt5_3 -->
 
 kt5_3 
 
 kt5_3 
 
<!-- kt1_2 -->
 
 kt1_2 
 
 kt1_2 
 
<!-- kt1_2--kt3_1 -->
 
 kt1_2--kt3_1 
 
 
<!-- kt1_2--kt3_3 -->
 
 kt1_2--kt3_3 
 
 
<!-- kt1_2--kt2_0 -->
 
 kt1_2--kt2_0 
 
 
<!-- kt2_0--kt3_2 -->
 
 kt2_0--kt3_2 
 
 
<!-- kt2_0--kt4_1 -->
 
 kt2_0--kt4_1 
 
 
<!-- kt2_2--kt4_3 -->
 
 kt2_2--kt4_3 
 
 
<!-- kt2_2--kt3_0 -->
 
 kt2_2--kt3_0 
 
 
<!-- kt2_2--kt4_1 -->
 
 kt2_2--kt4_1 
 
 
<!-- kt4_2 -->
 
 kt4_2 
 
 kt4_2 
 
<!-- kt4_2--kt5_0 -->
 
 kt4_2--kt5_0 
 
 
<!-- kt5_1 -->
 
 kt5_1 
 
 kt5_1 
 
<!-- kt4_3--kt5_1 -->
 
 kt4_3--kt5_1 
 
 
<!-- kt2_3--kt3_1 -->
 
 kt2_3--kt3_1 
 
 
<!-- kt2_3--kt4_2 -->
 
 kt2_3--kt4_2 
 
 
<!-- kt3_2--kt4_0 -->
 
 kt3_2--kt4_0 
 
 
<!-- kt3_2--kt5_3 -->
 
 kt3_2--kt5_3 
 
 
<!-- kt3_2--kt5_1 -->
 
 kt3_2--kt5_1 
 
 
<!-- kt0_0 -->
 
 kt0_0 
 
 kt0_0 
 
<!-- kt0_0--kt1_2 -->
 
 kt0_0--kt1_2 
 
 
<!-- kt2_1 -->
 
 kt2_1 
 
 kt2_1 
 
<!-- kt0_0--kt2_1 -->
 
 kt0_0--kt2_1 
 
 
<!-- kt0_2 -->
 
 kt0_2 
 
 kt0_2 
 
<!-- kt0_2--kt1_0 -->
 
 kt0_2--kt1_0 
 
 
<!-- kt0_2--kt2_3 -->
 
 kt0_2--kt2_3 
 
 
<!-- kt0_2--kt2_1 -->
 
 kt0_2--kt2_1 
 
 
<!-- kt1_3--kt3_2 -->
 
 kt1_3--kt3_2 
 
 
<!-- kt1_3--kt2_1 -->
 
 kt1_3--kt2_1 
 
 
<!-- kt0_3 -->
 
 kt0_3 
 
 kt0_3 
 
<!-- kt0_3--kt1_1 -->
 
 kt0_3--kt1_1 
 
 
<!-- kt0_3--kt2_2 -->
 
 kt0_3--kt2_2 
 
 
<!-- kt3_0--kt4_2 -->
 
 kt3_0--kt4_2 
 
 
<!-- kt3_0--kt5_1 -->
 
 kt3_0--kt5_1 
 
 
<!-- kt4_1--kt5_3 -->
 
 kt4_1--kt5_3 
 
 
<!-- kt2_1--kt3_3 -->
 
 kt2_1--kt3_3 
 
 
<!-- kt2_1--kt4_0 -->
 
 kt2_1--kt4_0 
 
 
<!-- kt2_1--kt4_2 -->
 
 kt2_1--kt4_2

-----

## Hamiltonian path finding

- Finding a Hamiltonian path is NP-complete problem. 
- Hence, using algorithms that is fast, but do not always "work" can be desirable.

Two algorithms to consider:

- The backtracking algorithm with [Warnsdorf's rule](https://en.wikipedia.org/wiki/Knight%27s_tour#Warnsdorf's_rule)
- Random (Angluin-Valiant) algorithm

Using (deterministic) backtracking algorithm to find a Hamiltonian path:

In [9]:
my $end = 'kt0_3'; #'kt0_3';# Whatever;
my @hpath = $gkt.find-hamiltonian-path('kt0_0', $end, method => 'backtracking', :warnsdorf-rule);

[kt0_0 kt1_2 kt2_0 kt0_1 kt1_3 kt3_2 kt5_3 kt4_1 kt3_3 kt5_2 kt4_0 kt2_1 kt0_2 kt1_0 kt2_2 kt3_0 kt5_1 kt4_3 kt3_1 kt5_0 kt4_2 kt2_3 kt1_1 kt0_3]

Using (random) Angluin-Valiant algorithm:

In [10]:
my @hpath = $gkt.find-hamiltonian-path('kt0_0', $end, method => 'random', pick => 'max-degree');

[kt0_0 kt2_1 kt1_3 kt0_1 kt2_0 kt1_2 kt3_3 kt4_1 kt5_3 kt3_2 kt4_0 kt5_2 kt3_1 kt5_0 kt4_2 kt3_0 kt5_1 kt4_3 kt2_2 kt1_0 kt0_2 kt2_3 kt1_1 kt0_3]

Show the found Hamiltonian Knight path on a chessboard:

In [11]:
#% html

# Reusing the "chessboard graph" DOT spec above.
my $preamble = [
    $dot.subst('--', '->', :g).subst('label="",').subst('Chessboard by DOT', "Knight's Hamiltonian path"),
    'node [shape=ellipse, fillcolor=Yellow, width=0.6, height=0.2, fixedsize=true, style=filled, color=Orange, fontsize=10, fontcolor=Black];',
    'edge [style=filled, width=103, color=Orange];',
].join("\n");

# Make a path graph from the found Hamiltonian path.
my $gp = Graph::Path.new(@hpath):directed;

# Set the graph vertex coordinates.
$gp.vertex-coordinates = $gkt.vertex-coordinates;

# Replace (rename) the path graph vertices to ordinals. 
# (In order to show the sequential order of the path.)
$gp .= vertex-replace((@hpath Z=> (^@hpath.elems)».Str).Hash); 

$gp.dot(:$preamble, engine=>'neato'):svg;

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
 Knight's Hamiltonian path 
<!-- 0_2 -->
 
 0_2 
 
 0_2 
 
<!-- 0_3 -->
 
 0_3 
 
 0_3 
 
<!-- 0_2->0_3 -->
<!-- 1_2 -->
 
 1_2 
 
 1_2 
 
<!-- 0_2->1_2 -->
<!-- 2_4 -->
 
 2_4 
 
 2_4 
 
<!-- 3_4 -->
 
 3_4 
 
 3_4 
 
<!-- 2_4->3_4 -->
<!-- 2_5 -->
 
 2_5 
 
 2_5 
 
<!-- 2_4->2_5 -->
<!-- 0_4 -->
 
 0_4 
 
 0_4 
 
<!-- 0_5 -->
 
 0_5 
 
 0_5 
 
<!-- 0_4->0_5 -->
<!-- 1_4 -->
 
 1_4 
 
 1_4 
 
<!-- 0_4->1_4 -->
<!-- 2_2 -->
 
 2_2 
 
 2_2 
 
<!-- 3_2 -->
 
 3_2 
 
 3_2 
 
<!-- 2_2->3_2 -->
<!-- 2_3 -->
 
 2_3 
 
 2_3 
 
<!-- 2_2->2_3 -->
<!-- 1_5 -->
 
 1_5 
 
 1_5 
 
<!-- 1_5->2_5 -->
<!-- 3_3 -->
 
 3_3 
 
 3_3 
 
<!-- 3_2->3_3 -->
<!-- 3_1 -->
 
 3_1 
 
 3_1 
 
<!-- 3_1->3_2 -->
<!-- 2_1 -->
 
 2_1 
 
 2_1 
 
<!-- 2_1->2_2 -->
<!-- 2_1->3_1 -->
<!-- 0_5->1_5 -->
<!-- 2_3->2_4 -->
<!-- 2_3->3_3 -->
<!-- 3_0 -->
 
 3_0 
 
 3_0 
 
<!-- 3_0->3_1 -->
<!-- 0_3->0_4 -->
<!-- 1_3 -->
 
 1_3 
 
 1_3 
 
<!-- 0_3->1_3 -->
<!-- 0_0 -->
 
 0_0 
 
 0_0 
 
<!-- 0_1 -->
 
 0_1 
 
 0_1 
 
<!-- 0_0->0_1 -->
<!-- 1_0 -->
 
 1_0 
 
 1_0 
 
<!-- 0_0->1_0 -->
<!-- 1_4->2_4 -->
<!-- 1_4->1_5 -->
<!-- 3_5 -->
 
 3_5 
 
 3_5 
 
<!-- 0_1->0_2 -->
<!-- 1_1 -->
 
 1_1 
 
 1_1 
 
<!-- 0_1->1_1 -->
<!-- 3_4->3_5 -->
<!-- 2_0 -->
 
 2_0 
 
 2_0 
 
<!-- 2_0->2_1 -->
<!-- 2_0->3_0 -->
<!-- 1_0->2_0 -->
<!-- 1_0->1_1 -->
<!-- 1_2->2_2 -->
<!-- 1_2->1_3 -->
<!-- 1_3->2_3 -->
<!-- 1_3->1_4 -->
<!-- 1_1->2_1 -->
<!-- 1_1->1_2 -->
<!-- 3_3->3_4 -->
<!-- 2_5->3_5 -->
<!-- 12 -->
 
 12 
 
 12 
 
<!-- 13 -->
 
 13 
 
 13 
 
<!-- 12->13 -->
 
 12->13 
 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 3->4 -->
 
 3->4 
 
 
 
<!-- 22 -->
 
 22 
 
 22 
 
<!-- 23 -->
 
 23 
 
 23 
 
<!-- 22->23 -->
 
 22->23 
 
 
 
<!-- 19 -->
 
 19 
 
 19 
 
<!-- 20 -->
 
 20 
 
 20 
 
<!-- 19->20 -->
 
 19->20 
 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 6->7 -->
 
 6->7 
 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- 10->11 -->
 
 10->11 
 
 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 8->9 -->
 
 8->9 
 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 5->6 -->
 
 5->6 
 
 
 
<!-- 11->12 -->
 
 11->12 
 
 
 
<!-- 4->5 -->
 
 4->5 
 
 
 
<!-- 18 -->
 
 18 
 
 18 
 
<!-- 18->19 -->
 
 18->19 
 
 
 
<!-- 14 -->
 
 14 
 
 14 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 14->15 -->
 
 14->15 
 
 
 
<!-- 17 -->
 
 17 
 
 17 
 
<!-- 17->18 -->
 
 17->18 
 
 
 
<!-- 21 -->
 
 21 
 
 21 
 
<!-- 21->22 -->
 
 21->22 
 
 
 
<!-- 9->10 -->
 
 9->10 
 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0->1 -->
 
 0->1 
 
 
 
<!-- 20->21 -->
 
 20->21 
 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 2->3 -->
 
 2->3 
 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 15->16 -->
 
 15->16 
 
 
 
<!-- 16->17 -->
 
 16->17 
 
 
 
<!-- 7->8 -->
 
 7->8 
 
 
 
<!-- 1->2 -->
 
 1->2 
 
 
 
<!-- 13->14 -->
 
 13->14

------

## References

### Articles

[Wk1] Wikipedia entry, ["Knight's tour"](https://en.wikipedia.org/wiki/Knight%27s_tour).

[Wk2] Wikipedia entry, ["Hamiltonian path problem"](https://en.wikipedia.org/wiki/Hamiltonian_path_problem).

### Packages, paclets

[AAp1] Anton Antonvov,
[Graph Raku package](https://github.com/antononcube/Raku-Graph),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonvov,
[JavaScript::D3 Raku package](https://github.com/antononcube/Raku-JavaScript-D3),
(2022-2024),
[GitHub/antononcube](https://github.com/antononcube).

### Videos

[AAv1] Anton Antonov,
[Graph neat examples in Raku (Set 1)](https://youtu.be/5qXgqqRZHow),
(2024),
[YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).

[AAv2] Anton Antonov,
[Graph neat examples in Raku (Set 2)](https://youtu.be/E7qhutQcWCY),
(2024),
[YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).

[AAv3] Anton Antonov,
[Graph neat examples in Raku (Set 3)](),
(2024),
[YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).
